In [6]:
from warnings import warn
from pathlib import Path

from pandas import read_csv
from arcpy import Describe

from esri_file_to_dataframe import esri_to_df

poi_fc = r'I:\Projects\Darren\PPA3_GIS\PPA3_GIS.gdb\POI_overture_2025'  # r'I:\Projects\Darren\PPA3_GIS\SHP\POI_data\POI.shp'


def make_poi_tables(in_fc, output_dir):
    target_sref = 'EPSG:4326' # spatial ref used by conveyal
    wt_csv = 'poi_wts.csv'
    df_wts = read_csv(wt_csv)
    f_poicat = 'category'
    f_ppacat = 'cat_ppa'
    f_destcnt = 'dest_cnt'
    
    fc_sref = Describe(in_fc).spatialReference.name
    df = esri_to_df(in_fc, include_geom=True, crs_val=fc_sref)
    df.to_crs(target_sref, inplace=True)
    df['x'] = df.geometry.x
    df['y'] = df.geometry.y
    df = df[[f_poicat, 'x', 'y']]

    df = df.merge(df_wts, on=f_poicat, how='left')

    df_unmatched = df.loc[df[f_destcnt].isnull()] # poi categories that weren't found in the weights table
    if df.loc[df[f_destcnt].isnull()].shape[0] > 0:
        unmatched_cats = df_unmatched[f_poicat].value_counts()  # .drop_duplicates().values
        warn(f"""The following POI categories are not in {wt_csv} and will not be counted in results:
        {unmatched_cats}""")
        # import pdb; pdb.set_trace()

    df = df.loc[~df[f_destcnt].isnull()] # only keep categories that have a lookup value in the poi_wts CSV

    # make separate csvs for each category you want to get accessibility numbers for
    out_cols = ['x', 'y']
    for pcat in df_wts[f_ppacat].drop_duplicates().values:
        catcol = f"{pcat}_dests"
        df[f"{pcat}_dests"] = 0 # default value
        df.loc[df[f_ppacat] == pcat, catcol] = df[f_destcnt]

        out_cols.append(catcol)

    outpath = Path(output_dir).joinpath(f"ppa_poi.csv")
    df[out_cols].to_csv(outpath, index=False)
    print(f"result in {outpath}")

result_dir = r'I:\Projects\Darren\PPA3_GIS\CSV\poi'
make_poi_tables(poi_fc, result_dir)

result in I:\Projects\Darren\PPA3_GIS\CSV\poi\ppa_poi.csv


C:\Users\dconly\AppData\Local\Temp\ipykernel_15704\3505653989.py:32: UserWarning: The following POI categories are not in poi_wts.csv and will not be counted in results:
        category
corporate_office    1
gas_station         1
Name: count, dtype: int64
  warn(f"""The following POI categories are not in {wt_csv} and will not be counted in results:
